In [13]:
import pymysql
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC  
from sklearn.model_selection import train_test_split
from sklearn import cross_validation,metrics
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, roc_auc_score, classification_report, roc_curve, auc  

'''
conn = pymysql.connect(host='192.168.1.131',user='yichen', password='0000', database='yelp_db', charset='utf8')
cur = conn.cursor()
query="select id, review_count,useful,funny,cool,fans,average_stars, compliment_hot+ compliment_more+ compliment_profile+ compliment_cute+ compliment_list+ compliment_note+ compliment_plain+compliment_cool+compliment_funny+compliment_writer+compliment_photos as sum_compliment from user limit 50000"
cur.execute(query)
t=cur.fetchall()
user_info_50000=pd.DataFrame(list(t), columns = [i[0] for i in cur.description])
print('done')
'''

'\nconn = pymysql.connect(host=\'192.168.1.131\',user=\'yichen\', password=\'0000\', database=\'yelp_db\', charset=\'utf8\')\ncur = conn.cursor()\nquery="select id, review_count,useful,funny,cool,fans,average_stars, compliment_hot+ compliment_more+ compliment_profile+ compliment_cute+ compliment_list+ compliment_note+ compliment_plain+compliment_cool+compliment_funny+compliment_writer+compliment_photos as sum_compliment from user limit 50000"\ncur.execute(query)\nt=cur.fetchall()\nuser_info_50000=pd.DataFrame(list(t), columns = [i[0] for i in cur.description])\nprint(\'done\')\n'

In [2]:
#query2='select review.id as review_id, business_id,user_id,stars, review.useful as review_useful,review.funny as review_funny, review.cool as review_cool, user_info_50000.review_count as user_review_count, user_info_50000.useful as user_useful,user_info_50000.fun as user_fun,user_info_50000.cool as user_cool,user_info_50000.fans as user_fans,user_info_50000.averagestars as user_averagestars,user_info_50000.sum_compliment as user_sum_compliment from review inner join user_info_50000 on review.user_id=user_info_50000.id'
#user_info['avgOpinionsPerReview']=(user_info['useful']+user_info['funny']+user_info['cool'])/user_info['review_count']
#user_info_50000
query2='select id,business_id,user_id,stars,useful,funny,cool from review'
cur.execute(query2)
tt=cur.fetchall()
review_reviewer_50000=pd.DataFrame(list(tt), columns = [i[0] for i in cur.description])
print('done')

done


In [25]:
query3='select * from category'
cur.execute(query3)
cat=cur.fetchall()
category=pd.DataFrame(list(cat), columns = [i[0] for i in cur.description])
print('done')

done


In [87]:
#rename column name in case mix up
user_info_50000.columns=['user_id','user_review_count','user_useful','user_funny','user_cool','user_fans','user_average_stars','user_sum_compliment']
#review=review_reviewer_50000
#review.columns = ['review_id', 'business_id','user_id','stars','review_useful''review_funny','review_cool']
#review_user_50000=pd.merge(review,user_info_50000,on='user_id',how='inner')

In [33]:
review_cat_user_50000=pd.merge(category,review_user_50000,on='business_id',how='inner')

In [95]:
user_cat_count=review_cat_user_50000.groupby(['user_id','category'], as_index=False).count()
user_cat_sum=review_cat_user_50000.groupby(['user_id','category'], as_index=False).sum()


In [99]:
user_cat_count=user_cat_count[['user_id','category','review_id']]
user_cat_count.rename(columns={'review_id':'review_count'}, inplace=True)

E:\Tools\Anaconda3\lib\site-packages\pandas\core\frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [112]:
user_cat_stat=pd.merge(user_cat_count,user_cat_sum,on=['user_id','category'],how='inner')

In [25]:
#review_cat_user_50000

In [133]:
#del user_cat_stat['user_review_count']
#del user_cat_stat['user_useful']
#del user_cat_stat['user_funny']
#del user_cat_stat['user_cool']
#del user_cat_stat['user_fans']
#del user_cat_stat['user_sum_compliment']
#del user_cat_stat['user_average_stars']
user_cat_stat['stars']=user_cat_stat['stars']/user_cat_stat['review_count']
user_cat_stat['review_useful']=user_cat_stat['review_useful']/user_cat_stat['review_count']
user_cat_stat['review_funny']=user_cat_stat['review_funny']/user_cat_stat['review_count']
user_cat_stat['review_cool']=user_cat_stat['review_cool']/user_cat_stat['review_count']
user_cat_stat.rename(columns={'review_count':'uc_review_count','stars':'uc_avg_stars','review_useful':'uc_avg_review_useful','review_funny':'uc_avg_review_funny','review_cool':'uc_avg_review_cool'}, inplace=True)
user_cat_stat

,user_id,category,uc_review_count,uc_avg_stars,uc_avg_review_useful,uc_avg_review_funny,uc_avg_review_cool
0,---1lKK3aKOuomHnwAkAow,Accessories,3,3.333333,1.666667,0.666667,0.333333
1,---1lKK3aKOuomHnwAkAow,Active Life,9,3.777778,2.555556,0.555556,1.333333
2,---1lKK3aKOuomHnwAkAow,Adult Education,1,5.000000,1.000000,1.000000,1.000000
3,---1lKK3aKOuomHnwAkAow,American (New),16,3.687500,2.000000,0.437500,0.937500
4,---1lKK3aKOuomHnwAkAow,American (Traditional),5,4.000000,2.000000,0.400000,1.000000
5,---1lKK3aKOuomHnwAkAow,Amusement Parks,2,3.500000,3.500000,0.000000,2.500000
6,---1lKK3aKOuomHnwAkAow,Art Classes,1,5.000000,5.000000,0.000000,2.000000
7,---1lKK3aKOuomHnwAkAow,Art Galleries,2,3.500000,4.000000,1.500000,1.500000
8,---1lKK3aKOuomHnwAkAow,Art Supplies,1,5.000000,5.000000,0.000000,2.000000
9,---1lKK3aKOuomHnwAkAow,Arts & Crafts,3,4.000000,4.000000,1.000000,2.333333


In [135]:
review_cat_user_userFeature_50000=pd.merge(review_cat_user_50000,user_cat_stat,on=['user_id','category'],how='inner')

In [26]:
#review_cat_user_userFeature_50000


In [10]:
#review_cat_user_50000.loc[(review_cat_user_50000['user_id']=='---1lKK3aKOuomHnwAkAow') & (review_cat_user_50000['category']=='Accessories')]

In [2]:
cur.execute('select * from category')
t=cur.fetchall()
category=pd.DataFrame(list(t), columns = [i[0] for i in cur.description])

In [7]:

train, test = train_test_split(review_cat_user_userFeature_50000, test_size=0.2)
train_rev_lst = [[] for _ in range(len(train))]
train_lab_lst = []
    
test_rev_lst = [[] for _ in range(len(test))]
test_lab_lst = []
print('start')
#user_review_count	user_useful	user_funny	user_cool	user_fans	user_average_stars	user_sum_compliment	uc_review_count	uc_avg_stars
#uc_avg_review_useful	uc_avg_review_funny	uc_avg_review_coo
ini = 0
for index, row in train.iterrows():
    #print(ini, len(train), ini/len(train))
    train_rev_lst[ini].extend((row['review_useful'],row['review_funny'],
                   row['review_cool'],row['user_review_count'],row['user_useful'],
                   row['user_funny'],row['user_cool'],row['user_fans'],
                   row['user_average_stars'],row['user_sum_compliment'],row['uc_review_count'],
                   row['uc_avg_stars'],row['uc_avg_review_useful'],row['uc_avg_review_funny'],row['uc_avg_review_cool']))
    if row['review_stars'] > 3:
        train_lab_lst.append(1)
    else:
        train_lab_lst.append(0)
    ini += 1 

print('training features completed')
ini = 0
for index, row in test.iterrows():
    #print(ini, len(test), ini/len(test))
    test_rev_lst[ini].extend((row['review_useful'],row['review_funny'],
                   row['review_cool'],row['user_review_count'],row['user_useful'],
                   row['user_funny'],row['user_cool'],row['user_fans'],
                   row['user_average_stars'],row['user_sum_compliment'],row['uc_review_count'],
                   row['uc_avg_stars'],row['uc_avg_review_useful'],row['uc_avg_review_funny'],row['uc_avg_review_cool']))
    if row['review_stars'] > 3:
        test_lab_lst.append(1)
    else:
        test_lab_lst.append(0)
    
    ini += 1
print('testing features completed')   


logreg = linear_model.LogisticRegression(C=1e5)
logreg.fit(train_rev_lst, train_lab_lst)

prepro = logreg.predict_proba(test_rev_lst)
acc = logreg.score(test_rev_lst,test_lab_lst)

print('the accuracy of logistic regression is %f' %acc)  

test_auc = metrics.roc_auc_score(logreg.predict(test_rev_lst),test_lab_lst)#验证集上的auc值
print('the auc of logistic regression is %f' %test_auc)

print(metrics.confusion_matrix(logreg.predict(test_rev_lst),test_lab_lst))#验证集上的混淆矩阵

'''    
clf = SVC().fit(train_rev_lst, train_lab_lst) 

res = clf.predict(test_rev_lst) 

accu = 0

for i in range(len(test_rev_lst)):
    #print(i, len(test_rev_lst),i/len(test_rev_lst))
    if test_lab_lst[i] == res[i]:    
        accu += 1

accu = accu / len(test_rev_lst)

print('the accuracy is %f' %accu)  

print('the auc is %f' %metrics.roc_auc_score(test_lab_lst,res))

print(metrics.confusion_matrix(test_lab_lst,res))
'''

start
training features completed
testing features completed
the accuracy of logistic regression is 0.799679
the auc of logistic regression is 0.846049
[[ 26366   2449]
 [ 32792 114316]]


"    \nclf = SVC().fit(train_rev_lst, train_lab_lst) \n\nres = clf.predict(test_rev_lst) \n\naccu = 0\n\nfor i in range(len(test_rev_lst)):\n    #print(i, len(test_rev_lst),i/len(test_rev_lst))\n    if test_lab_lst[i] == res[i]:    \n        accu += 1\n\naccu = accu / len(test_rev_lst)\n\nprint('the accuracy is %f' %accu)  \n\nprint('the auc is %f' %metrics.roc_auc_score(test_lab_lst,res))\n\nprint(metrics.confusion_matrix(test_lab_lst,res))\n"

In [24]:
'''
print('Start RF')
model_RF = RandomForestClassifier(n_estimators=70, min_samples_split=10, min_samples_leaf=10,max_depth=22,max_features='sqrt',random_state=10,class_weight = 'balanced')
x_train_RF = train_rev_lst
y_train_RF = train_lab_lst
x_test_RF = test_rev_lst
y_test_RF = test_lab_lst
model_RF.fit(x_train_RF,y_train_RF)
pre_RF = model_RF.predict_proba(x_test_RF)
'''
#fpr, tpr, thresholds = roc_curve(y_test_RF, pre_RF[:,1])  
#roc_auc = auc(fpr, tpr)  
pre_lab_RF=model_RF.predict(x_test_RF)
t=precision_score(test_lab_lst,pre_lab_RF)
print(t,roc_auc)


0.938698904253 0.96304366877


In [ ]:
model_Ada = AdaBoostClassifier( base_estimator = RandomForestClassifier(n_estimators=70, min_samples_split=10, min_samples_leaf=10,max_depth =22,max_features='sqrt' ,random_state=10,class_weight = 'balanced'),
                        algorithm="SAMME",
                        n_estimators=200, learning_rate=0.1)
x_train_Ada = train_rev_lst
y_train_Ada = train_lab_lst
x_test_Ada = test_rev_lst
model_Ada.fit(x_train_Ada,y_train_Ada)
pre_Ada = model_Ada.predict_proba(x_test_Ada)

fpr, tpr, thresholds = roc_curve(test_lab_lst, pre_Ada[:,1])  
roc_auc = auc(fpr, tpr)  


pre_lab_RF=model_Ada.predict(x_test_RF)
t=precision_score(test_lab_lst,pre_lab_RF)
print(t,roc_auc)

In [ ]:
import lightgbm as lgb
### GBM models-----------------------------------------------------------------------------------------  
lgb_train = lgb.Dataset(train_rev_lst, train_lab_lst)
lgb_eval = lgb.Dataset(test_rev_lst, test_lab_lst, reference=lgb_train)

        # specify your configurations as a dict
params = {
            'task': 'predict',
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': {'l2', 'auc'},
            'num_leaves': 91,
            'learning_rate': 0.1,
            'feature_fraction': 0.9,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'verbose': 0
}

       # print('Start training...')
        # train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

        #print('Save model...')
        # save model to file
gbm.save_model('model.txt')
pre_GBM = gbm.predict(test_rev_lst)
gbm.fit(train_rev_lst, test_rev_lst)
fpr, tpr, thresholds = roc_curve(y_test, pre_GBM)  
roc_auc = auc(fpr, tpr)  
print(roc_auc)

the accuracy of logistic regression is 0.799679, the auc of logistic regression is 0.846049  
confusion matrix for test set:  
[[ 26366   2449]  
 [ 32792 114316]]  
 

In [ ]:
query2='create table review_cat as (select review.business_id,user_id,stars,useful,funny,cool,category from category right join review on category.business_id=review.business_id)'
cur.execute(query2)
cur.execute('commit')
#t2=cur.fetchall()
#review=pd.DataFrame(list(t2), columns = [i[0] for i in cur.description])

In [8]:
#review_cat_user_userFeature_50000=pd.read_csv('review_cat_user_userFeature.csv')
